In [ ]:
import os
import numpy as np
import pandas as pd
import librosa

#声優の声データからMFCC（声の特徴量）を抽出し、表にまとめる。

X_data = []  # 特徴行列
y_data = []  # クラスラベルデータ
speakers_number = 7 # 声優の人数
files_number = 100 # 一人あたりのサンプルの数

for speaker_num in range(1, speakers_number + 1):  # 声優フォルダの数だけ繰り返し処理
    # ボイスサンプルがあるフォルダ名
    dir_name = f'VoiceSamples/{str(speaker_num).zfill(3)}'
    file_list = os.listdir(dir_name)
    for i in range(files_number):
        file_path = os.path.join(dir_name, file_list[i])  # 音声ファイルへのパス
        y, sr = librosa.load(file_path)  # 音声ファイルを読み込む
        mfcc = librosa.feature.mfcc(y, sr)  # MFCC
        mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
        mfcc = mfcc.flatten()
        mfcc = mfcc.tolist()
        mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
        X_data.append(mfcc)
        y_data.append(speaker_num)

X = pd.DataFrame(X_data, columns=[f'mfcc_{n}' for n in range(1, 13)]) # mfcc_1,mfcc_2,..のように行のタイトルをつける
y = pd.DataFrame({'target': y_data}) # targetタイトルを追加

df = pd.concat([X, y], axis=1) # 合体！！！！！！
df.to_csv('mfcc.csv', index=False)  # csvで保存
df.head() # 見せてほしいな

In [ ]:

import pandas as pd
from pycaret.regression import *

# 機械学習前の下準備を行う。以下、Pycaretの関数を使う場合このセルを実行してから使うこと

mfcc_data = pd.read_csv("mfcc.csv") # MFCCのデータを読み込む
# データの前準備(以下、Pycaretの関数を使う場合setupを呼び出してから使うこと)
reg = setup(data=mfcc_data, target='target', data_split_shuffle=True, use_gpu=True, silent=True, fold=5, n_jobs=-1)


In [ ]:
# 機械学習のモデルを複数比較し、精度が高かったモデルを表示する。

compare_models()

In [15]:
from pycaret.regression import *
# 最も精度が高かったモデルを生成

model = create_model('et') # モデルを生成（）
#tuned_model = tune_model(model) # モデルを調整...すると精度落ちるの何...しないほうがいいのか？
predict_data = predict_model(model) # 既存のデータで予測
predict_data.to_csv('predict_result.csv', index=False)  # csvで保存


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.1473,1.9948,1.4124,0.5042,0.3792,0.6227
1,1.2014,2.3295,1.5263,0.4368,0.3457,0.4675
2,1.0217,1.7889,1.3375,0.5353,0.2780,0.3223
3,1.1822,2.3691,1.5392,0.3496,0.3393,0.3989
4,1.0508,1.8213,1.3496,0.5464,0.3229,0.4456
Mean,1.1207,2.0607,1.4330,0.4745,0.3330,0.4514
Std,0.0717,0.2461,0.0854,0.0732,0.0331,0.0991


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,1.0134,1.6936,1.3014,0.5689,0.3122,0.4253


In [16]:
final_model = finalize_model(model)
save_model(final_model,'Final Model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='target',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy...
                  ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                    

In [6]:
from pycaret.regression import *

final_model = load_model('Final Model') # モデルの読み込み

Transformation Pipeline and Model Successfully Loaded


In [20]:
import sounddevice as sd
import wave
import numpy as np

#音声を録音

FILE_NAME = 'my_recording.wav'  # 保存するファイル名
wave_length = 5  # 録音する長さ（秒）
sample_rate = 16_000  # サンプリング周波数

print("録音開始")
# 録音開始（wave_length秒間録音。wait で録音し終わるまで待つ）
data = sd.rec(int(wave_length * sample_rate), sample_rate, channels=1)
sd.wait()
print("録音終了")

# ノーマライズ。量子化ビット16bitで録音するので int16 の範囲で最大化する
data = data / data.max() * np.iinfo(np.int16).max

# float -> int
data = data.astype(np.int16)

# ファイル保存
with wave.open(FILE_NAME, mode='wb') as wb:
    wb.setnchannels(1)  # モノラル
    wb.setsampwidth(2)  # 16bit=2byte
    wb.setframerate(sample_rate)
    wb.writeframes(data.tobytes())  # バイト列に変換


録音開始
録音終了


In [4]:
import pandas as pd
import librosa
import numpy as np

###MFCCを抽出###

data = []
TEST_FILE_NAME = "test.wav"

y, sr = librosa.load(TEST_FILE_NAME)  # 音声ファイルを読み込む
mfcc = librosa.feature.mfcc(y, sr)  # MFCC
mfcc = np.average(mfcc, axis=1)  # 時間平均を取る
mfcc = mfcc.flatten()
mfcc = mfcc.tolist()
mfcc = mfcc[1:13]  # 低次の係数を取り出す（12次まで取り出すことが多い）
data.append(mfcc)

df = pd.DataFrame(data, columns=[f'mfcc_{n}' for n in range(1, 13)])

df.to_csv('mfcc_my_recording.csv', index=False)  # csvで保存
df.head()

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12
0,61.791355,-18.600464,18.558226,-1.541007,-2.50288,0.355464,-0.349908,-7.095498,1.180879,-10.101668,4.389037,-3.882792


In [7]:
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
from pycaret.regression import *

#録音した音声データからどの声優に似ているかを特定

actors = ["下野紘", "花江夏樹", "梶裕貴", "沢城みゆき", "鬼頭明里", "水瀬いのり", "悠木碧"]

predict_my_recording = predict_model(final_model, data = pd.read_csv("mfcc_my_recording.csv")) # 未知データを予測
predict_label = predict_my_recording.at[0, "Label"] # 予測データを取得（小数点数）
voice_actor_number = Decimal(str(predict_label)).quantize(Decimal('0'), rounding=ROUND_HALF_UP) # 整数に四捨五入
print("あなたの声は" + actors[int(voice_actor_number-1)] + "に似ています。")
print("予測値：" + str(predict_label))

あなたの声は鬼頭明里に似ています。
予測値：2.66
